# import dataframe

In [1]:
# importer les bibliothèques importantes
import pandas as pd
import numpy as np
import datetime
from datetime import timedelta

In [2]:
name_basics = pd.read_csv('https://datasets.imdbws.com/name.basics.tsv.gz', sep='\t', low_memory=False)

In [3]:
title_akas = pd.read_csv("https://datasets.imdbws.com/title.akas.tsv.gz", sep='\t', low_memory=False)

In [4]:
title_basics = pd.read_csv("https://datasets.imdbws.com/title.basics.tsv.gz", sep='\t', low_memory=False)

In [5]:
title_crew = pd.read_csv("https://datasets.imdbws.com/title.crew.tsv.gz", sep='\t', low_memory=False)

In [6]:
title_episode = pd.read_csv("https://datasets.imdbws.com/title.episode.tsv.gz", sep='\t', low_memory=False)

In [6]:
title_principals = pd.read_csv("https://datasets.imdbws.com/title.principals.tsv.gz", sep='\t', low_memory=False)

In [7]:
title_ratings = pd.read_csv("https://datasets.imdbws.com/title.ratings.tsv.gz", sep='\t', low_memory=False)

# Merge and clean dataframe

In [ ]:

'''Dans le code qui suit, nous allons faire un premier clean des dataset de base,
ainsi que des jointures pour alléger le dataframe que nous allons utiliser pour la suite'''

# Dans la table title_basics, ne garder que les films
title_basics = title_basics[title_basics['titleType'].str.contains('movie', na=False, regex=False)]

# Mettre la colonne startYear en format date
title_basics['startYear'] = pd.to_datetime(title_basics['startYear'], infer_datetime_format=True, errors='coerce')

# Ne garder que les lignes dont la date de sortie est supérieure à 2000
title_basics = title_basics[title_basics['startYear'] >= pd.to_datetime('2000-01-01')]

# Dans la colonne genres, supprimer les valeurs nulles, les films pour adultes, et les documentaires
title_basics = title_basics[(title_basics['genres'].notnull()) & (title_basics['genres'].apply(lambda val: all(s not in str(val) for s in ['Adult', 'Documentary', '\\N'])))]

# Retirer toutes les lignes qui n'ont pas de durée
title_basics = title_basics[(title_basics['runtimeMinutes'].notnull()) & (~title_basics['runtimeMinutes'].str.contains('\\N', na=False, regex=False))]

# Joindre title_basics et title_akas en df9
df9 = title_basics.merge(title_akas, left_on=['tconst'], right_on=['titleId'], how='left', suffixes=['_title_basics', '_title_akas'])

# Renommer df9 en merged_title_akas
merged_basics_akas = df9

# Joindre title_crew et title_principals en df6
temp_df = title_principals.drop_duplicates(subset=['tconst']) # Supprimer les doublons et ne merger que la première occurence
df6 = title_crew.merge(temp_df, left_on=['tconst'], right_on=['tconst'], how='left', suffixes=['_title_crew', '_title_principals'])

# Joindre title_crew et title principals en df7
df7 = title_crew.merge(title_principals, left_on=['tconst'], right_on=['tconst'], how='left', suffixes=['_title_crew', '_title_principals'])

# Joindre title_crew et name_basics en df8
df8 = title_crew.merge(name_basics, left_on=['tconst'], right_on=['nconst'], how='left', suffixes=['_title_crew', '_name_basics'])

# Joindre title_crew et name_basics en df9
df9 = title_crew.merge(name_basics, left_on=['directors'], right_on=['nconst'], how='left', suffixes=['_title_crew', '_name_basics'])

# Joindre title_crew et name_basics en df10
title_crew_tmp = title_crew.drop(['writers'], axis=1)
df10 = title_crew_tmp.merge(name_basics, left_on=['directors'], right_on=['nconst'], how='left', suffixes=['_title_crew', '_name_basics'])

# Joindre title_crew et name_basics en df11
title_crew_tmp = title_crew.drop(['writers'], axis=1)
name_basics_tmp = name_basics.drop(['deathYear'], axis=1)
df11 = title_crew_tmp.merge(name_basics_tmp, left_on=['directors'], right_on=['nconst'], how='left', suffixes=['_title_crew', '_name_basics'])

# Joindre title_crew et name_basics en df12
title_crew_tmp = title_crew.drop(['writers'], axis=1)
name_basics_tmp = name_basics.drop(['birthYear', 'deathYear'], axis=1)
df12 = title_crew_tmp.merge(name_basics_tmp, left_on=['directors'], right_on=['nconst'], how='left', suffixes=['_title_crew', '_name_basics'])

# Joindre title_crew et name_basics en df13
title_crew_tmp = title_crew.drop(['writers'], axis=1)
name_basics_tmp = name_basics.drop(['birthYear', 'deathYear', 'knownForTitles'], axis=1)
df13 = title_crew_tmp.merge(name_basics_tmp, left_on=['directors'], right_on=['nconst'], how='left', suffixes=['_title_crew', '_name_basics'])

# Joindre df12 et df11 en df14
temp_df = df11.drop_duplicates(subset=['tconst']) # Supprimer les doublons et ne merger que la première occurence
df14 = df12.merge(temp_df, left_on=['tconst'], right_on=['tconst'], how='left', suffixes=['_df12', '_df11'])

# Joindre merged_basics_akas et df11 en df15
temp_df = df11.drop_duplicates(subset=['tconst']) # Supprimer les doublons et ne merger que la première occurence
df15 = merged_basics_akas.merge(temp_df, left_on=['tconst'], right_on=['tconst'], how='left', suffixes=['_merged_basics_akas', '_df11'])

# Joindre merged_basics_akas et df11 en df16
df16 = merged_basics_akas.merge(df11, left_on=['tconst'], right_on=['tconst'], how='left', suffixes=['_merged_basics_akas', '_df11'])

# Joindre merged_basics_akas et df12 en df17
df17 = merged_basics_akas.merge(df12, left_on=['tconst'], right_on=['tconst'], how='left', suffixes=['_merged_basics_akas', '_df12'])

# Joindre merged_basics_akas et df12 en df18
df12_tmp = df12.drop(['knownForTitles'], axis=1)
df18 = merged_basics_akas.merge(df12_tmp, left_on=['tconst'], right_on=['tconst'], how='left', suffixes=['_merged_basics_akas', '_df12'])

# Joindre merged_basics_akas et df12 en df19
df12_tmp = df12.drop(['primaryProfession', 'knownForTitles'], axis=1)
df19 = merged_basics_akas.merge(df12_tmp, left_on=['tconst'], right_on=['tconst'], how='left', suffixes=['_merged_basics_akas', '_df12'])

# Supprimer la colonne titleType
df19.drop(['titleType'], axis=1, inplace=True)

# Supprimer la colonne isAdult
df19.drop(['isAdult'], axis=1, inplace=True)

# Supprimer la colonne endYear
df19.drop(['endYear'], axis=1, inplace=True)

# Supprimer la colonne titleId
df19.drop(['titleId'], axis=1, inplace=True)

# Supprimer la colonne ordering
df19.drop(['ordering'], axis=1, inplace=True)

# Ne garder que les lignes dont la région est FR
df19 = df19[(df19['region'].notnull()) & (df19['region'].str.contains('FR', na=False, regex=False)) & (~df19['region'].str.contains('\\N', na=False, regex=False))]

# Supprimer la colonne language
df19.drop(['language'], axis=1, inplace=True)

# Supprimer la colonne types
df19.drop(['types'], axis=1, inplace=True)

# Supprimer la colonne attributes
df19.drop(['attributes'], axis=1, inplace=True)

# Supprimer la colonne isOriginalTitle
df19.drop(['isOriginalTitle'], axis=1, inplace=True)

# Ne garder que les données non-nulles de la colonne primaryName
df19 = df19[df19['primaryName'].notnull()]

# Joindre df19 et title_ratings en df20
df20 = df19.merge(title_ratings, left_on=['tconst'], right_on=['tconst'], how='left', suffixes=['_df19', '_title_ratings'])

# Ne garder que les données non-nulles de la colonne numVotes
df20 = df20[df20['numVotes'].notnull()]

# Ne garder que les données non-nulles de la colonne primaryName
df20 = df20[df20['primaryName'].notnull()]

# Ne garder que acteurs et actrices dans la colonne category
title_principals = title_principals[title_principals['category'].str.contains('act', na=False, regex=False)]

# Joindre title_principals et title_ratings en df7
temp_df = title_ratings.drop_duplicates(subset=['tconst']) # Supprimer les doublons et ne merger que la première occurence
df7 = title_principals.merge(temp_df, left_on=['tconst'], right_on=['tconst'], how='left', suffixes=['_title_principals', '_title_ratings'])

# Joindre title_principals et name_basics en df8
temp_df = name_basics.drop_duplicates(subset=['nconst']) # Supprimer les doublons et ne merger que la première occurence
df8 = title_principals.merge(temp_df, left_on=['nconst'], right_on=['nconst'], how='left', suffixes=['_title_principals', '_name_basics'])

# Joindre title_principals et name_basics en df9
df9 = title_principals.merge(name_basics, left_on=['nconst'], right_on=['nconst'], how='left', suffixes=['_title_principals', '_name_basics'])

# Supprimer la colonne birthYear
name_basics_tmp = name_basics.drop(['birthYear'], axis=1)

# Joindre title_principals et name_basics en df10
df10 = title_principals.merge(name_basics_tmp, left_on=['nconst'], right_on=['nconst'], how='left', suffixes=['_title_principals', '_name_basics'])

# Renommer la colonne averageRating en Note_moyenne
df20.rename(columns={'averageRating': 'Note_moyenne'}, inplace=True)

# Renommer la colonne numVotes en Nb_votes
df20.rename(columns={'numVotes': 'Nb_votes'}, inplace=True)

# Renommer la colonne primaryName en Realisateur
df20.rename(columns={'primaryName': 'Realisateur'}, inplace=True)

# Joindre df20 et df7 en df11
temp_df = df7.drop_duplicates(subset=['tconst']) # Supprimer les doublons et ne merger que la première occurence
df11 = df20.merge(temp_df, left_on=['tconst'], right_on=['tconst'], how='left', suffixes=['_df20', '_df7'])

# Joindre df20 et df10 en df12
df10_tmp = df10.drop(['deathYear', 'nconst', 'primaryName', 'category', 'knownForTitles', 'job', 'characters', 'primaryProfession', 'ordering'], axis=1)
df12 = df20.merge(df10_tmp, left_on=['tconst'], right_on=['tconst'], how='left', suffixes=['_df20', '_df10'])

# Joindre df20 et df10 en df13
df10_tmp = df10.drop(['deathYear', 'nconst', 'category', 'knownForTitles', 'job', 'characters', 'primaryProfession', 'ordering'], axis=1)
df13 = df20.merge(df10_tmp, left_on=['tconst'], right_on=['tconst'], how='left', suffixes=['_df20', '_df10'])

# Supprimer la colonne nconst
df13.drop(['nconst'], axis=1, inplace=True)

# Supprimer la colonne directors
df13.drop(['directors'], axis=1, inplace=True)

# Ne garder que les données non-nulles de la colonne primaryTitle
df13 = df13[df13['primaryTitle'].notnull()]

# Ne garder que les données non-nulles de la colonne originalTitle
df13 = df13[df13['originalTitle'].notnull()]

# Ne garder que les données non-nulles de la colonne startYear
df13 = df13[df13['startYear'].notnull()]

# Ne garder que les données non-nulles de la colonne runtimeMinutes
df13 = df13[df13['runtimeMinutes'].notnull()]

# Ne garder que les données non-nulles de la colonne genres
df13 = df13[df13['genres'].notnull()]

# Ne garder que les données non-nulles de la colonne title
df13 = df13[df13['title'].notnull()]

# Ne garder que les données non-nulles de la colonne region
df13 = df13[df13['region'].notnull()]

# Ne garder que les données non-nulles de la colonne Realisateur
df13 = df13[df13['Realisateur'].notnull()]

# Ne garder que les données non-nulles de la colonne Note_moyenne
df13 = df13[df13['Note_moyenne'].notnull()]

# Ne garder que les données non-nulles de la colonnne Nb_votes
df13 = df13[df13['Nb_votes'].notnull()]

# Ne garder que les données non-nulles de la colonne primaryName
df13 = df13[df13['primaryName'].notnull()]

# Supprimer la colonne primaryTitle
df13.drop(['primaryTitle'], axis=1, inplace=True)

# Renommer la colonne originalTitle en Titre original
df13.rename(columns={'originalTitle': 'Titre_original'}, inplace=True)

# Renommer la colonne startYear en Annee
df13.rename(columns={'startYear': 'Annee'}, inplace=True)

# Renommer la colonne runtimeMinutes en Duree
df13.rename(columns={'runtimeMinutes': 'Duree'}, inplace=True)

# Renommer la colonne genres en Genres
df13.rename(columns={'genres': 'Genres'}, inplace=True)

# Renommer la colonne title en Titre
df13.rename(columns={'title': 'Titre'}, inplace=True)

# Supprimer la colonne region
df13.drop(['region'], axis=1, inplace=True)

# Renommer la colonne primaryName en Acteurs
df13.rename(columns={'primaryName': 'Acteurs'}, inplace=True)


'''exporter le fichier en csv local via mitosheet'''

# ouvrir le fichier csv sauvegardé en local pour terminer le cleaning
films_db = pd.read_csv("C:/Users/tit_r/Desktop/Formation Data Analyst/Projets/Projet 2/Dataset/MitoExport1.csv")

# ne garder que les films dont la durée est supérieure à 50 minutes et inférieure à 240 minutes
films_db=films_db.loc[(films_db['Duree'] >= 50) & (films_db['Duree'] <= 240)]

# créer une liste des acteurs pour chaque film
films_db['liste_acteurs'] = films_db.groupby('tconst')['Acteurs'].transform(lambda x: ', '.join(x))

# supprimer les lignes en double pour les films qui apparaissent plusieurs fois
films_db = films_db.drop_duplicates(subset=['tconst'], keep='first')

# supprimer la colonne acteurs
films_db = films_db.drop(['Acteurs'], axis=1)

# télécharger le fichier cleané en csv local
films_db = films_db.to_csv('C:/Users/tit_r/Desktop/Formation Data Analyst/Projets/Projet 2/Dataset/films_db.csv', index=False)

# Joindre le dataset cleané avec le csv imdb pour  rajouter des infos

In [26]:
# ouvrir le fichier json pour récupérer les infos
import json
with open("tt_movies.json", "r") as f:
    data = json.load(f) # lecture du fichier
    
#[key, poster_path, tagline, overview, popularity]
df_n = []

# Boucle for pour parcourir toutes les clés
for key in data:
    try:
        
        poster_path = data[key]["poster_path"]
        tagline = data[key]["tagline"]
        overview = data[key]["overview"]
        popularity = data[key]["popularity"]
        poster_path = data[key]["poster_path"]
        
        provisoire = [key, tagline, overview, popularity,poster_path] # list provisoire
        
        df_n.append(provisoire) # on ajoute a list
        
    except KeyError: # si try erreur
        provisoire = [key, None, None, None,None] # on ajoute quand meme la ligne mais vide
        df_n.append(provisoire)
        
r = pd.DataFrame(df_n,columns=['tconst','tagline','overview','popularity','poster_path'])
r

,tconst,tagline,overview,popularity,poster_path
0,tt0035423,"If they lived in the same century, they'd be p...",When her scientist ex-boyfriend discovers a po...,15.452,/mUvikzKJJSg9khrVdxK8kg3TMHA.jpg
1,tt0069049,40 years in the making.,"Surrounded by fans and skeptics, grizzled dire...",10.437,/kFky1paYEfHxfCYByEc9g7gn6Zk.jpg
2,tt0026246,I am Sonya! You don't know who or what I am......,A man is haunted by a murder he's committed.,2.972,/baF1oPKQdMxpiMdJijVNu2c8Yz1.jpg
3,tt0086722,None,None,NaN,None
4,tt0118694,"Feel the heat, keep the feeling burning, let t...",Two neighbors become intimate after discoverin...,34.211,/iYypPT4bhqXfq1b6EnmxvRt6b2Y.jpg
...,...,...,...,...,...
19373,tt0674687,None,None,NaN,None
19374,tt0341445,,"In Africa in the fifties, during the Mau-Mau w...",1.400,/pkF81kdo2k3Wf34H4DAzFSaHr2o.jpg
19375,tt9911196,,Jan has been married to Gedda for 35 years. Ge...,3.436,/qZNlF6i3MOvwhRV8SDfp5M40hVx.jpg
19376,tt0017250,,After a daredevil demonstration of aviator stu...,0.600,None


In [28]:
# exporter en csv le fichier qui contient les infos supplémentaires
r.to_csv('films_more_columns.csv', index=False)

In [32]:
# ouvrir le fichier films_db
films_db2 = pd.read_csv('C:/Users/tit_r/Desktop/Formation Data Analyst/Projets/Projet 2/Dataset/films_db2.csv')

In [30]:
# ouvrir le fichier qui contient les infos supplémentaires
test = pd.read_csv('films_more_columns.csv')

In [31]:
test

,tconst,tagline,overview,popularity,poster_path
0,tt0035423,"If they lived in the same century, they'd be p...",When her scientist ex-boyfriend discovers a po...,15.452,/mUvikzKJJSg9khrVdxK8kg3TMHA.jpg
1,tt0069049,40 years in the making.,"Surrounded by fans and skeptics, grizzled dire...",10.437,/kFky1paYEfHxfCYByEc9g7gn6Zk.jpg
2,tt0026246,I am Sonya! You don't know who or what I am......,A man is haunted by a murder he's committed.,2.972,/baF1oPKQdMxpiMdJijVNu2c8Yz1.jpg
3,tt0086722,NaN,NaN,NaN,NaN
4,tt0118694,"Feel the heat, keep the feeling burning, let t...",Two neighbors become intimate after discoverin...,34.211,/iYypPT4bhqXfq1b6EnmxvRt6b2Y.jpg
...,...,...,...,...,...
19373,tt0674687,NaN,NaN,NaN,NaN
19374,tt0341445,NaN,"In Africa in the fifties, during the Mau-Mau w...",1.400,/pkF81kdo2k3Wf34H4DAzFSaHr2o.jpg
19375,tt9911196,NaN,Jan has been married to Gedda for 35 years. Ge...,3.436,/qZNlF6i3MOvwhRV8SDfp5M40hVx.jpg
19376,tt0017250,NaN,After a daredevil demonstration of aviator stu...,0.600,NaN


In [ ]:
from mitosheet.public.v3 import *; register_analysis("id-bkahhhgjxc");

# Joindre films_db et test en df3
df3 = films_db2.merge(test, left_on=['tconst'], right_on=['tconst'], how='left', suffixes=['_films_db', '_test'])

# Renommer la colonne tagline en Slogan
df3.rename(columns={'tagline': 'Slogan'}, inplace=True)

# Renommer la colonne overview en Resume
df3.rename(columns={'overview': 'Resume'}, inplace=True)

# Renommer la colonne popularity en Note_imdb
df3.rename(columns={'popularity': 'Note_imdb'}, inplace=True)

# Renommer la colonne poster path en Jaquette
df3.rename(columns={'poster_path': 'Jaquette'}, inplace=True)

# exporter le fichier en local via mitosheet

In [53]:
# importer le fichier csv joint

films_db3 = pd.read_csv('C:/Users/tit_r/Desktop/Formation Data Analyst/Projets/Projet 2/Dataset/MitoExport3.csv')

In [28]:
films_db3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22126 entries, 0 to 22125
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   tconst          22126 non-null  object 
 1   Titre_original  22126 non-null  object 
 2   Annee           22126 non-null  object 
 3   Duree           22126 non-null  int64  
 4   Genres          22126 non-null  object 
 5   Titre           22126 non-null  object 
 6   Realisateur     22126 non-null  object 
 7   Note_moyenne    22126 non-null  float64
 8   Nb_votes        22126 non-null  float64
 9   liste_acteurs   22126 non-null  object 
 10  Slogan          5263 non-null   object 
 11  Resume          12883 non-null  object 
 12  Note_imdb       13161 non-null  float64
 13  Jaquette        12891 non-null  object 
dtypes: float64(3), int64(1), object(10)
memory usage: 2.4+ MB


In [9]:
films_db3.head()

,tconst,Titre_original,Annee,Duree,Genres,Titre,Realisateur,Note_moyenne,Nb_votes,liste_acteurs,Slogan,Resume,Note_imdb,Jaquette
0,tt0035423,Kate & Leopold,2001-01-01,118,"Comedy,Fantasy,Romance",Kate et Léopold,James Mangold,6.4,86502.0,"Meg Ryan, Hugh Jackman, Liev Schreiber, Brecki...","If they lived in the same century, they'd be p...",When her scientist ex-boyfriend discovers a po...,15.452,/mUvikzKJJSg9khrVdxK8kg3TMHA.jpg
1,tt0069049,The Other Side of the Wind,2018-01-01,122,Drama,De l'autre côté du vent,Orson Welles,6.7,7643.0,"John Huston, Oja Kodar, Peter Bogdanovich, Sus...",40 years in the making.,"Surrounded by fans and skeptics, grizzled dire...",10.437,/kFky1paYEfHxfCYByEc9g7gn6Zk.jpg
2,tt0096056,Crime and Punishment,2002-01-01,126,Drama,Crime et Châtiment,Menahem Golan,5.6,834.0,"Crispin Glover, Vanessa Redgrave, John Hurt, M...",NaN,NaN,NaN,NaN
3,tt0118589,Glitter,2001-01-01,104,"Drama,Music,Romance",Glitter,Vondie Curtis-Hall,2.4,23736.0,"Mariah Carey, Eric Benét, Max Beesley, Da Brat",NaN,NaN,NaN,NaN
4,tt0118694,Fa yeung nin wah,2000-01-01,98,"Drama,Romance",In the Mood for Love,Kar-Wai Wong,8.1,155632.0,"Tony Leung Chiu-wai, Maggie Cheung, Siu Ping-L...","Feel the heat, keep the feeling burning, let t...",Two neighbors become intimate after discoverin...,34.211,/iYypPT4bhqXfq1b6EnmxvRt6b2Y.jpg


# Mettre à jour les jaquettes à récupérer

In [39]:
# créer une variable pour ne garder que les films pour lesquels nous n'avons pas le resume

tt_resume_null = films_db3.loc[films_db3['Resume'].isnull()]

In [40]:
# exporter le fichier csv

tt_resume_null = tt_resume_null.to_csv('C:/Users/tit_r/Desktop/Formation Data Analyst/Projets/Projet 2/Dataset/tt_resume_null.csv', index=False)

In [54]:
# récupérer les données manquantes mises à jour
donnees_manquantes_1 = pd.read_csv('C:/Users/tit_r/Desktop/Formation Data Analyst/Projets/Projet 2/Dataset/donnees_manquantes_1.csv')

In [55]:
# récupérer les données manquantes mises à jour
donnees_manquantes_2 = pd.read_csv('C:/Users/tit_r/Desktop/Formation Data Analyst/Projets/Projet 2/Dataset/donnees_manquantes_suite_2.csv')

In [56]:
# récupérer les données manquantes mises à jour
donnees_manquantes_3 = pd.read_csv('C:/Users/tit_r/Desktop/Formation Data Analyst/Projets/Projet 2/Dataset/donnees_manquantes_suite_3.csv')

In [57]:
# concaténer les 3 csv avec des données manquantes

df_n  = pd.concat([donnees_manquantes_1,donnees_manquantes_2,donnees_manquantes_3])

df_n = df_n.dropna(subset=['poster_path'])

df_n.rename(columns={'tagline' : ''})

df_n.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9243 entries, 0 to 10
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Unnamed: 0   9243 non-null   int64  
 1   tconst       9243 non-null   object 
 2   tagline      4977 non-null   object 
 3   overview     8897 non-null   object 
 4   popularity   9243 non-null   float64
 5   poster_path  9243 non-null   object 
dtypes: float64(1), int64(1), object(4)
memory usage: 505.5+ KB


In [8]:
df_n.head()

,Unnamed: 0,tconst,tagline,overview,popularity,poster_path
0,0,tt0096056,NaN,A modern day adaptation of Dostoyevsky's class...,3.803,/2ckMQwDi11TofiNoaE3sHrYbaCh.jpg
1,1,tt0118589,"In music she found her dream, her love, herself.","A young woman is catapulted into pop stardom, ...",10.111,/388ru0R7fnBFPaVuJe4caRGGh54.jpg
2,2,tt0119866,NaN,"On the island of Porquerolles, Alice spies on ...",0.621,/fkL2ejKu8QcPu8TYcU8vyo7cIE1.jpg
3,3,tt0120166,NaN,"Every hundred years, the evil sorceress Morgan...",5.942,/x1ZDLPgXlPvULLTlskjesrOZ9nP.jpg
4,4,tt0120667,4 times the action. 4 times the adventure. 4 t...,"During a space voyage, four scientists are alt...",28.341,/8HLQLILZLhDQWO6JDpvY6XJLH75.jpg


In [58]:
# renommer les colonnes pour faciliter le merge avec le dataframe de base 

df_n.drop('Unnamed: 0', axis=1, inplace=True)

df_n.rename(columns={'tagline':'Slogan', 'overview':'Resume', 'popularity':'Note_imdb', 'poster_path':'Jaquette'},
            inplace=True)

In [34]:
df_n.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9243 entries, 0 to 10
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   tconst     9243 non-null   object 
 1   Slogan     4977 non-null   object 
 2   Resume     8897 non-null   object 
 3   Note_imdb  9243 non-null   float64
 4   Jaquette   9243 non-null   object 
dtypes: float64(1), object(4)
memory usage: 433.3+ KB


In [35]:
# concaténer le dataframe de base avec le dataframe qui contient les infos de films
# films_db3.set_index('tconst', inplace=True)

# df_n.set_index('tconst', inplace=True)

In [61]:
films_db3.set_index('tconst', inplace=True)
df_n.set_index('tconst', inplace=True)

for i in films_db3.index:
    if i in df_n.index:
        films_db3.loc[i, 'Slogan'] = df_n.loc[i, 'Slogan'] if pd.notnull(df_n.loc[i, 'Slogan']) else films_db3.loc[i, 'Slogan']
        films_db3.loc[i, 'Resume'] = df_n.loc[i, 'Resume'] if pd.notnull(df_n.loc[i, 'Resume']) else films_db3.loc[i, 'Resume']
        films_db3.loc[i, 'Note_imdb'] = df_n.loc[i, 'Note_imdb'] if pd.notnull(df_n.loc[i, 'Note_imdb']) else films_db3.loc[i, 'Note_imdb']
        films_db3.loc[i, 'Jaquette'] = df_n.loc[i, 'Jaquette'] if pd.notnull(df_n.loc[i, 'Jaquette']) else films_db3.loc[i, 'Jaquette']

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().